In [48]:
import pandas as pd
import numpy as np

In [49]:
df = pd.read_csv('data/2011-cabi-trip-history-data-final.csv', parse_dates=['start_date','end_date'])

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127471 entries, 0 to 1127470
Data columns (total 7 columns):
start_date            1127471 non-null datetime64[ns]
end_date              1127471 non-null datetime64[ns]
duration_in_sec       1127471 non-null int64
start_station_name    1127471 non-null object
end_station_name      1127471 non-null object
bike#                 1127471 non-null object
member_type           1127471 non-null object
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 60.2+ MB


__ 1. Count the number of departure from each station in every month, store the result in df_depart __

In [51]:
df_depart = df.groupby([df.start_station_name,df.start_date.dt.month]).count()

In [52]:
df_depart.index.names = ['start_station','month']

In [53]:
df_depart.reset_index(inplace=True)

In [54]:
df_depart.drop(['end_date','duration_in_sec','end_station_name','bike#','member_type'], axis=1, inplace=True)

In [55]:
df_depart.rename(columns = {'start_date':'count_depart'},inplace=True)

In [56]:
df_depart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1370 entries, 0 to 1369
Data columns (total 3 columns):
start_station    1370 non-null object
month            1370 non-null int64
count_depart     1370 non-null int64
dtypes: int64(2), object(1)
memory usage: 32.2+ KB


In [57]:
df_depart.head(12)

,start_station,month,count_depart
0,10th & Monroe St NE,1,62
1,10th & Monroe St NE,2,86
2,10th & Monroe St NE,3,83
3,10th & Monroe St NE,4,168
4,10th & Monroe St NE,5,251
5,10th & Monroe St NE,6,227
6,10th & Monroe St NE,7,229
7,10th & Monroe St NE,8,230
8,10th & Monroe St NE,9,216
9,10th & Monroe St NE,10,207


__ 2. Count the number of arrival to each station in every month, store the result in df_arr __

In [58]:
df_arr = df.groupby([df.end_station_name,df.end_date.dt.month]).count()

In [59]:
df_arr.index.names = ['end_station','month']

In [60]:
df_arr.reset_index(inplace=True)

In [61]:
df_arr.drop(['end_date','duration_in_sec','start_station_name','bike#','member_type'], axis=1, inplace=True)

In [62]:
df_arr.rename(columns = {'start_date':'count_arr'},inplace=True)

In [63]:
df_arr.head(12)

,end_station,month,count_arr
0,10th & Monroe St NE,1,53
1,10th & Monroe St NE,2,73
2,10th & Monroe St NE,3,76
3,10th & Monroe St NE,4,149
4,10th & Monroe St NE,5,224
5,10th & Monroe St NE,6,196
6,10th & Monroe St NE,7,229
7,10th & Monroe St NE,8,218
8,10th & Monroe St NE,9,192
9,10th & Monroe St NE,10,171


In [64]:
df_arr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1374 entries, 0 to 1373
Data columns (total 3 columns):
end_station    1374 non-null object
month          1374 non-null int64
count_arr      1374 non-null int64
dtypes: int64(2), object(1)
memory usage: 32.3+ KB


__ 3. Merge the df_depart and df_arr __

In [65]:
df_combined = pd.merge(df_arr, df_depart, left_on = ['end_station','month'], right_on = ['start_station','month'], how= 'outer')

In [66]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1376 entries, 0 to 1375
Data columns (total 5 columns):
end_station      1374 non-null object
month            1376 non-null int64
count_arr        1374 non-null float64
start_station    1370 non-null object
count_depart     1370 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 64.5+ KB


In [67]:
df_combined = df_combined.fillna(0)

In [68]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1376 entries, 0 to 1375
Data columns (total 5 columns):
end_station      1376 non-null object
month            1376 non-null int64
count_arr        1376 non-null float64
start_station    1376 non-null object
count_depart     1376 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 64.5+ KB


In [72]:
def fill_name(row):
    if row['end_station'] ==0 :
        row['end_station'] = row['start_station']
    if row['start_station'] ==0:
        row['start_station'] = row['end_station']
    return row

df_combined = df_combined.apply(fill_name, axis=1)

In [73]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1376 entries, 0 to 1375
Data columns (total 5 columns):
end_station      1376 non-null object
month            1376 non-null int64
count_arr        1376 non-null float64
start_station    1376 non-null object
count_depart     1376 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 64.5+ KB


In [75]:
df_combined.rename(columns = {'end_station':'station'},inplace=True)

In [77]:
df_combined.head()

,station,month,count_arr,start_station,count_depart
0,10th & Monroe St NE,1,53.0,10th & Monroe St NE,62.0
1,10th & Monroe St NE,2,73.0,10th & Monroe St NE,86.0
2,10th & Monroe St NE,3,76.0,10th & Monroe St NE,83.0
3,10th & Monroe St NE,4,149.0,10th & Monroe St NE,168.0
4,10th & Monroe St NE,5,224.0,10th & Monroe St NE,251.0


In [80]:
df_combined.drop('start_station',axis=1,inplace=True)

In [81]:
df_combined.head()

,station,month,count_arr,count_depart
0,10th & Monroe St NE,1,53.0,62.0
1,10th & Monroe St NE,2,73.0,86.0
2,10th & Monroe St NE,3,76.0,83.0
3,10th & Monroe St NE,4,149.0,168.0
4,10th & Monroe St NE,5,224.0,251.0


In [83]:
df_combined['arr-depart'] = df_combined.count_arr-df_combined.count_depart

In [84]:
df_combined.head()

,station,month,count_arr,count_depart,arr-depart
0,10th & Monroe St NE,1,53.0,62.0,-9.0
1,10th & Monroe St NE,2,73.0,86.0,-13.0
2,10th & Monroe St NE,3,76.0,83.0,-7.0
3,10th & Monroe St NE,4,149.0,168.0,-19.0
4,10th & Monroe St NE,5,224.0,251.0,-27.0


In [85]:
df_combined.to_csv('data/2011-arr-depart-rate.csv')